In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [3]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pyknon
!{sys.executable} -m pip install music21
!{sys.executable} -m pip install nltk
!{sys.executable} -m pip install twython

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [31]:
from pyknon.genmidi import Midi
from pyknon.music import NoteSeq, Note
import random
from music21 import *
from urllib.parse import urlparse
from IPython.core.display import display, HTML
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

In [101]:
chromatic = ["A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#",
             "A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#",
            "A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#"]

In [6]:
def make_midi(notes, filename):
	notes1 = map(Note, notes)
	midi = Midi(1, tempo=90)
	midi.seq_notes(notes1, track = 0)
	midi.write(filename)

In [7]:
def play_midi(filename):
	mf = midi.MidiFile()
	mf.open(filename)
	mf.read()
	mf.close()
	s = midi.translate.midiFileToStream(mf)
	s.show('midi')

In [8]:
def compose_progression_from_word(word):
	result = []
	for char in word.upper():
		if char in ["A","B","C","D","E","F","G"]:
			result.append(char)
	if result:
		return result
	else:
		return random.sample(chromatic, 4)

In [566]:
#First, a progression is created calling the previous function. 
#Second, the triad is created by drawing a random sample of the chromatic scale.
#The compound sentiment polarity is computed and outputed as the second element of the tuple
    #positive sentiment: compound score >= 0.05
    #neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
    #negative sentiment: compound score <= -0.05
    # LEXICON https://github.com/cjhutto/vaderSentiment/blob/master/vaderSentiment/vader_lexicon.txt
    
    
def compose_chord_base(word, meaning, string=False):
   
    progression = compose_progression_from_word(word)
    harmony = stream.Stream()
    happyChords = []
    sadChords = []
    
    for base in progression:
        
        chord_string = base
        index = chromatic.index(base)
        
        
        # In order to have a consonant chord it's necessary to define the musical intervals
        tonic = chromatic[chromatic.index(chord_string.upper())]
        thirdMaj = chromatic[chromatic.index(chord_string.upper()) + 4]
        thirdMin =  chromatic[chromatic.index(chord_string.upper()) + 3]
        fifthPerf = chromatic[chromatic.index(chord_string.upper()) + 7]
        
        
        # Lexicon Sentiment Integration
        sid = SentimentIntensityAnalyzer()
        sentiment = sid.polarity_scores(meaning)
        
        
        # Sad chords creation
        if sentiment["compound"] >= 0.05:
            happyChords.append(chord.Chord([tonic,thirdMaj,fifthPerf]))
        else:
            sadChords.append(chord.Chord([tonic,thirdMin,fifthPerf]))

            
    # In order to have a musical harmony with more richness the quantity of notes in the word 
        # will be considered. If it's less than 3 notes other chords will be included at random in accordance with the
            # Sentiment.
    
    if len(word) <= 3 and sentiment["compound"] >= 0.05:      
        chord1 = random.sample(chromatic, 1)
        chord2 = random.sample(chromatic, 1)
        chord3 = random.sample(chromatic, 1)
        chord4 = random.sample(chromatic, 1)
        
        triadMaj1 = chord.Chord(
        [str(chord1[0]), 
         chromatic[chromatic.index(str(chord1[0])) + 4],  
         chromatic[chromatic.index(str(chord1[0])) + 7]])
        
        triadMaj2 = chord.Chord(
        [str(chord2[0]), 
         chromatic[chromatic.index(str(chord2[0])) + 4],  
         chromatic[chromatic.index(str(chord2[0])) + 7]])
        
        triadMaj3 = chord.Chord(
        [str(chord3[0]), 
         chromatic[chromatic.index(str(chord3[0])) + 4],  
         chromatic[chromatic.index(str(chord3[0])) + 7]])
        
        triadMaj4 = chord.Chord(
        [str(chord4[0]), 
         chromatic[chromatic.index(str(chord4[0])) + 4],  
         chromatic[chromatic.index(str(chord4[0])) + 7]])
        
        return(
            triadMaj1,triadMaj2,triadMaj3, triadMaj4)
    
    elif len(word) <= 3 and sentiment["compound"] < 0.05:
        chord1 = random.sample(chromatic, 1)
        chord2 = random.sample(chromatic, 1)
        chord3 = random.sample(chromatic, 1)
        chord4 = random.sample(chromatic, 1)
        
        triadMin1 = chord.Chord(
        [str(chord1[0]), 
         chromatic[chromatic.index(str(chord1[0])) + 3],  
         chromatic[chromatic.index(str(chord1[0])) + 7]])
        
        triadMin2 = chord.Chord(
        [str(chord2[0]), 
         chromatic[chromatic.index(str(chord2[0])) + 3],  
         chromatic[chromatic.index(str(chord2[0])) + 7]])
        
        triadMin3 = chord.Chord(
        [str(chord3[0]), 
         chromatic[chromatic.index(str(chord3[0])) + 3],  
         chromatic[chromatic.index(str(chord3[0])) + 7]])
        
        triadMin4 = chord.Chord(
        [str(chord4[0]), 
         chromatic[chromatic.index(str(chord4[0])) + 3],  
         chromatic[chromatic.index(str(chord4[0])) + 7]])
        
        return(triadMin1,triadMin2,triadMin3,triadMin4)
   
    elif len(happyChords) == 0:
        return(sadChords)
    else:
        return(happyChords)
    
        

In [567]:
compose_chord_base("abcd","I am very sad!")


[<music21.chord.Chord A C E>,
 <music21.chord.Chord B D F#>,
 <music21.chord.Chord C D# G>,
 <music21.chord.Chord D F A>]

In [10]:
def create_melody(word, meaning, string=False):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-10-ffe50f1f32b3>, line 2)

In [11]:
%%javascript
IPython.notebook.kernel.execute("URL = '" + window.location + "'");

<IPython.core.display.Javascript object>

In [19]:
try:
    word, meaning = str(urlparse(URL).query).replace("%20"," ").split("=")
except:
    word, meaning = "Water - Sky, Ocean, Beautiful".split(" - ")

In [34]:
chord_base = compose_chord_base(word, meaning)
chord_base

(<music21.stream.Stream 0x2b91d4842b0>, 0.5994)

In [26]:
display(HTML('<h1><center>' + f"{word} <br>{meaning}" + '</center></h1>'))
print("Chord Base:")
chord_base.show()

make_midi(chord_base, f"{word} - {meaning}.mid")

Chord Base:


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'C:\\Users\\Diego .LAPTOP-5GCM8JJT\\Dropbox\\AndtekVO\\rapsodia\\rapsodia\\C:\\Program Files (x86)\\MuseScore 2\\MuseScore.exe'

In [13]:
for chord in rapsodiapp:
    print(chord.isConsonant())

NameError: name 'rapsodiapp' is not defined

In [25]:
a = scale.MajorScale('a')
a.show()

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'C:\\Users\\Diego .LAPTOP-5GCM8JJT\\Dropbox\\AndtekVO\\rapsodia\\rapsodia\\C:\\Program Files (x86)\\MuseScore 2\\MuseScore.exe'